## 学习算法的实现
**前提**
神经网络存在合适的权重和偏置，调整权重和偏置以便拟合训练数据的过程成为学习。学习步骤分为四个步骤：

**STEP1(mini-batch)**
从训练数据中随机选出一部分数据，这部分数据成为mini-batch。我们的不表示减小mini-batch损失函数的值。

**STEP2计算梯度**
为了减小mini-batch损失函数的值，需要求出各个权重参数的梯度。梯度表示损失函数的值减小最多的方向。

**STEP3更新参数**
将权重参数沿梯度方向进行微小更新。

**STEP4重复**
重复步骤1，2，3

因为使用的数据是mini batch数据，所以称为“随机梯度下降法”（stochastic gradient descent， SGD）

## 实现手写数字识别的神经网络

In [1]:
import sys, os
sys.path.append(os.pardir)
from common.functions import *
from common.gradient import numerical_gradient

In [2]:
class TwoLayerNet:

    def __init__(self, input_size, hidden_size, output_size, weight_init_std=0.01):
        # 初始化权重
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)

    def predict(self, x):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
    
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        return y
        
    # x:输入数据, t:监督数据
    def loss(self, x, t):
        y = self.predict(x)
        
        return cross_entropy_error(y, t)
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        t = np.argmax(t, axis=1)
        
        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy
        
    # x:输入数据, t:监督数据
    def numerical_gradient(self, x, t):
        loss_W = lambda W: self.loss(x, t)
        
        grads = {}
        grads['W1'] = numerical_gradient(loss_W, self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W, self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W, self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W, self.params['b2'])
        
        return grads
        
    def gradient(self, x, t):
        W1, W2 = self.params['W1'], self.params['W2']
        b1, b2 = self.params['b1'], self.params['b2']
        grads = {}
        
        batch_num = x.shape[0]
        
        # forward
        a1 = np.dot(x, W1) + b1
        z1 = sigmoid(a1)
        a2 = np.dot(z1, W2) + b2
        y = softmax(a2)
        
        # backward
        dy = (y - t) / batch_num
        grads['W2'] = np.dot(z1.T, dy)
        grads['b2'] = np.sum(dy, axis=0)
        
        da1 = np.dot(dy, W2.T)
        dz1 = sigmoid_grad(a1) * da1
        grads['W1'] = np.dot(x.T, dz1)
        grads['b1'] = np.sum(dz1, axis=0)

        return grads

**参数说明**

input_size：输入层的神经元数量（即输入数据的特征维度）。

hidden_size：隐藏层的神经元数量。

output_size：输出层的神经元数量（如分类任务中的类别数）。

weight_init_std：权重初始化的标准差（默认值 0.01），控制初始权重的随机范围。



w1：输入层到隐藏层的权重矩阵，形状为 (input_size, hidden_size)。

使用 高斯分布（正态分布）随机初始化，乘以 weight_init_std 缩小数值范围，避免初始权重过大导致梯度不稳定。

例如：若 input_size=100，hidden_size=50，则 w1 是一个 100×50 的矩阵。

b1：隐藏层的偏置向量，形状为 (hidden_size,)。

初始化为全零向量（常见做法），因为偏置的初始值对模型影响较小。

In [3]:
net = TwoLayerNet(input_size=784, hidden_size=100, output_size=10)

In [4]:
x = np.random.rand(100, 784) # 伪输入数据
y = net.predict(x)

In [5]:
#x = np.random.rand(100, 784) # 伪输入数据
#t = np.random.rand(100, 10) # 伪正确解标签

#grads = net.gradient(x, t)
#print(grads)

In [6]:
#x = np.random.rand(100, 784) 
#t = np.random.rand(100, 10) 

#grads = net.numerical_gradient(x, t)
#print(grads)